# Аугументация датасета, состоящего из данных с сайтов: Проза.ру, ЛитПричал, Брифли, Литрес

In [120]:
import re

import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

nltk.download("punkt")

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
dataset = pd.read_csv("../data/temp_data/all_data.csv")

In [101]:
dataset.describe()

,title,text
count,31122,31101
unique,28839,30640
top,латырь-камень бел-горюч... сказка?,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [102]:
dataset.dropna(inplace=True)

In [103]:
dataset.describe()

,title,text
count,31101,31101
unique,28818,30640
top,из жизни графомана,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [104]:
dataset = dataset[dataset.text != ""].reset_index(drop=True)
dataset.describe()

,title,text
count,31101,31101
unique,28818,30640
top,из жизни графомана,; ; «энцикловикипедия»- -имена. посвящается ми...
freq,38,38


In [105]:
dataset[dataset.duplicated(keep=False)]

,title,text
28,латырь-камень бел-горюч... сказка?,домашняя диссертация любознательного обывателя...
40,на распутина мессмера серебряного века,"""распутин в кровавом поту молился и спасал"" пи..."
146,энцикловикипедия. топонимы,; «энцикловикипедия». топонимы. история соврем...
251,три поколения швондеров и мы выбираем...,"в повести булгакова ""собачье сердце"" есть домо..."
258,"просыпаюсь, а ты рядом! дню семьи","постоянное место публикации: сборник ""о любви..."
...,...,...
30682,таёжный глаз шерхана с гарью каина,"она заступалась за коллективного папика, подош..."
30871,энцикловикипедия. имена,; ; «энцикловикипедия»- -имена. посвящается ми...
30970,"просыпаюсь, а ты рядом! дню семьи","постоянное место публикации: сборник ""о любви..."
31011,корнеслов. история слов,; ; -история слов. «энцикловикипедия» посвящае...


In [106]:
dataset[dataset.text == "описание отсутствует"]

,title,text


In [107]:
dataset = dataset.drop_duplicates()

In [108]:
dataset.describe()

,title,text
count,30679,30679
unique,28818,30640
top,***,­
freq,20,19


In [109]:
dataset.loc[195].text

'\xad'

In [110]:
dataset = dataset[dataset.text != "\xad"].reset_index(drop=True)

In [52]:
dataset.describe()

,title,text
count,30660,30660
unique,28800,30639
top,***,"стихотворение ищите в рецензии, поскольку тата..."
freq,20,4


In [53]:
dataset = dataset[~dataset.duplicated(subset=["text"], keep=False)]

In [54]:
dataset.describe()

,title,text
count,30623,30623
unique,28763,30623
top,***,­­… сказки осенних вечеров ... монохромные во...
freq,20,1


In [56]:
dataset.reset_index(drop=True, inplace=True)

In [57]:
dataset

,title,text
0,время - октябрь...,­­… сказки осенних вечеров ... монохромные во...
1,переписка,"здравствуйте, вы установили мессенджер max, по..."
2,айвенго (спектакль) часть 3-я. ричард – львино...,радиоспектакль по роману.англия времен царство...
3,калевала,где можно найти наиболее полный литературный о...
4,янки из коннектикута при дворе короля артура,"новый, адаптированный перевод алексея козлова ..."
...,...,...
30618,что бывало,борис житков увлекался астрономией и фотографи...
30619,сон макара,"«этот сон видел бедный макар, который загнал с..."
30620,финал. золотое солнце правосудия,раннее утро предвещало жаркий день. солнечные ...
30621,святочные истории,святки – удивительный период времени: один год...


In [111]:
dup_titles = (
    dataset.groupby("text")["title"]
    .nunique()
    .reset_index()
    .query("title > 1")
)

print(f"Текстов с одинаковыми содержаниями, но разными названиями: {len(dup_titles)}")

Текстов с одинаковыми содержаниями, но разными названиями: 16


In [112]:
dataset = dataset.groupby("text").agg(lambda x: x.value_counts().index[0]).reset_index()
dataset.describe()

,text,title
count,30639,30639
unique,30639,28779
top,"как любому живому организму, организации нуж...",***
freq,1,20


In [113]:
def is_meaningful(text):
    return bool(re.search(r"[А-Яа-яA-Za-z0-9]", text))

def clean_leading_punct(text):
    return re.sub(r"^[^\wА-Яа-я0-9]+", "", text).strip()

In [114]:
dataset = dataset[dataset.text.apply(is_meaningful)].reset_index(drop=True)
dataset.describe()

,text,title
count,30634,30634
unique,30634,28774
top,"как любому живому организму, организации нуж...",***
freq,1,20


In [116]:
dataset = dataset[dataset.text.str.len() > 10]
dataset = dataset[dataset.text.str.split().str.len() > 5]

In [117]:
dataset.text = dataset.text.str.replace(r"[^\w\s,.!?-]", " ", regex=True)
dataset.text = dataset.text.str.replace(r"\s+", " ", regex=True).str.strip()

In [118]:
dataset.text = dataset.text.apply(clean_leading_punct)
dataset.describe()

,text,title
count,30533,30533
unique,30488,28685
top,адам бид дебютный роман знаменитой английской ...,***
freq,3,20


In [119]:
dataset.reset_index(drop=True, inplace=True)
dataset = dataset[~dataset.duplicated(subset=["text"], keep=False)].reset_index(drop=True)
dataset.describe()

,text,title
count,30444,30444
unique,30444,28601
top,"как любому живому организму, организации нужна...",***
freq,1,20


## Разделение на выборки

In [147]:
train_df, val_df = train_test_split(
    dataset, test_size=0.2, random_state=42, shuffle=True
)

In [149]:
len(train_df), len(val_df)

(24355, 6089)

In [148]:
val_df.to_csv("../data/val_df.csv", index=False)

## Аугментация тренировочной выборки

In [126]:
def split_text_into_chunks(text, sentences_per_chunk=3):
    sentences = sent_tokenize(text, language="russian")
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk])
        chunks.append(chunk)
    return chunks

In [127]:
from tqdm import tqdm

augmented_rows = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    chunks = split_text_into_chunks(row.text, sentences_per_chunk=3)
    for chunk in chunks:
        augmented_rows.append({"text": chunk, "title": row.title})

augmented_dataset = pd.DataFrame(augmented_rows)

100%|██████████| 24355/24355 [00:24<00:00, 1003.24it/s]


In [128]:
augmented_dataset.describe()

,text,title
count,1346212,1346212
unique,1318194,23154
top,. . .,чистый хозяин собственного мира
freq,247,13972


In [129]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.strip().str.len() > 10]
augmented_dataset = augmented_dataset[augmented_dataset.text != ". . ."]

max_per_title = 500
augmented_dataset = augmented_dataset.groupby("title").head(max_per_title).reset_index(drop=True)

print(augmented_dataset.title.value_counts().head(10))

title
пророку ев ап иоанну богослову чудотв!              500
тайна зеленой молнии                                500
если солнечные часы отстают…                        500
любовь или свобода 3: второй шанс                   500
привет от сведенборга                               500
азбука жизни книга 5 иначе жить нельзя!             500
прошёл все пути                                     500
легенды айстауна                                    500
дорога в никуда? часть 2. звёздные разборки.        500
священная миссия, или как ощипать падшего ангела    500
Name: count, dtype: int64


In [130]:
augmented_dataset.describe()

,text,title
count,868595,868595
unique,851894,23154
top,"это не значит, что остальным жителям планеты н...",пророку ев ап иоанну богослову чудотв!
freq,52,500


In [131]:
augmented_dataset = augmented_dataset.drop_duplicates().reset_index(drop=True)

In [132]:
augmented_dataset.describe()

,text,title
count,866772,866772
unique,851894,23154
top,если же кого-то заинтересует более подробно то...,пророку ев ап иоанну богослову чудотв!
freq,52,500


In [133]:
dup_titles = (
    augmented_dataset.groupby("text")["title"]
    .nunique()
    .reset_index()
    .query("title > 1")
)

print(f"Текстов с одинаковыми содержаниями, но разными названиями: {len(dup_titles)}")

Текстов с одинаковыми содержаниями, но разными названиями: 3630


In [134]:
augmented_dataset = augmented_dataset.groupby("text").agg(lambda x: x.value_counts().index[0]).reset_index()

In [74]:
augmented_dataset.describe()

,text,title
count,1282668,1282668
unique,1282668,28735
top,"как любому живому организму, организации нуж...",сто блюд из кошки
freq,1,1000


In [135]:
augmented_dataset = augmented_dataset[augmented_dataset.text.apply(is_meaningful)].reset_index(drop=True)

In [136]:
augmented_dataset.describe()

,text,title
count,851528,851528
unique,851528,23145
top,"! ! если ссылка неактивна, просто вставьте в а...",сильнее жизни (1-25)
freq,1,500


In [137]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.len() > 10]
augmented_dataset = augmented_dataset[augmented_dataset.text.str.split().str.len() > 5]

In [138]:
augmented_dataset.reset_index(inplace=True, drop=True)

In [139]:
augmented_dataset.describe()

,text,title
count,845871,845871
unique,845871,23142
top,"! ! если ссылка неактивна, просто вставьте в а...",вопреки всему (роман о суини тодде)
freq,1,500


In [140]:
augmented_dataset.text = augmented_dataset.text.str.replace(r"[^\w\s,.!?-]", " ", regex=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"\s+", " ", regex=True).str.strip()

In [141]:
augmented_dataset.describe()

,text,title
count,845871,845871
unique,845871,23142
top,"! ! если ссылка неактивна, просто вставьте в а...",вопреки всему (роман о суини тодде)
freq,1,500


In [142]:
augmented_dataset.text = augmented_dataset.text.apply(clean_leading_punct)

In [143]:
augmented_dataset.describe()

,text,title
count,845871,845871
unique,845859,23142
top,"она решила покарать меня за то, что я возвелич...",вопреки всему (роман о суини тодде)
freq,2,500


In [144]:
augmented_dataset = augmented_dataset[~augmented_dataset.duplicated(subset=["text"], keep=False)]

In [145]:
augmented_dataset.describe()

,text,title
count,845847,845847
unique,845847,23142
top,"если ссылка неактивна, просто вставьте в адрес...",без чувств. книга 1
freq,1,500


In [146]:
augmented_dataset.to_csv("../data/all_data_augmented/train_df.csv", index=False)